In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense
from tensorflow.keras.layers import Activation, BatchNormalization, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

import tensorflow_model_optimization as tfmot

## Load and preprocess training and testing dataset

In [2]:
data= pd.read_csv("res.csv") 

In [3]:
data.head()

,one,two,three,four,five,six,seven,eight,cls
0,87,78,98,71,94,100,89,64,0
1,83,77,101,66,88,91,82,58,0
2,76,75,98,65,90,94,88,55,0
3,87,78,100,70,97,94,89,73,0
4,82,80,105,69,94,99,95,69,0


In [4]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=10)

In [5]:
train_data = train_data.reset_index(drop=True)
train_features = train_data.copy()
train_labels = train_features.pop("cls")
test_data = test_data.reset_index(drop=True)
test_features = test_data.copy()
test_labels = test_features.pop("cls")

In [6]:
train_features

,one,two,three,four,five,six,seven,eight
0,69,28,77,72,44,80,26,51
1,91,102,237,104,87,60,62,52
2,77,61,59,10,1,125,155,69
3,88,79,117,76,103,92,87,52
4,71,56,76,68,158,91,54,48
...,...,...,...,...,...,...,...,...
571,75,3,60,44,134,95,84,53
572,91,134,204,48,2,60,63,48
573,54,2,157,148,199,95,56,63
574,84,70,81,109,150,94,68,55


In [7]:
mean, std = train_features.values.mean(axis=0), train_features.values.std(axis=0)

In [8]:
assert mean.shape == (8,)
assert std.shape == (8,)

In [9]:
print(f"{mean}\n{std}")

[ 81.27430556  60.69618056 139.90277778  87.78819444 101.45138889
  98.40972222  60.98611111  47.171875  ]
[29.69119596 38.05961679 67.94310012 39.25168432 56.3846019  39.34889579
 35.38674935 18.1955993 ]


## Load and process training and testing dataset

In [10]:
# Transfer to nparray
num_classes = 6
train_features = train_features.values.astype("float32")
train_labels_one_hot = to_categorical(train_labels, num_classes, dtype="float32")
test_features = test_features.values.astype("float32")
test_labels_one_hot = to_categorical(test_labels, num_classes, dtype="float32")

In [11]:
def normalize(x):
    # Normalize the value first
    x = (x - mean) / std
    
    # Map -1 ~ 1 to -127 ~ 127
    x = x * 127
    
    # Set up the limit bound
    x = np.where(x > 127, 127, x)
    x = np.where(x < -128, -128, x)
    
    return x

In [12]:
# Normalize the features
train_features = normalize(train_features)
test_features = normalize(test_features)

## Model define and create

In [13]:
model = Sequential()

# FC1
model.add(Input(shape=(8,), dtype=tf.float32))
model.add(Dense(64, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))

# FC2
model.add(Dense(128, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))

# FC3
model.add(Dense(6))

In [14]:
# Show your model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8192      
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 128)               0

## Model training

In [15]:
# Training model

# Define optimizer loss function and merics 
model.compile(
    optimizer="adam", loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"])

# Set training
model.fit(
    train_features, train_labels_one_hot, 
    validation_split=0.1, batch_size=64,
    verbose=1, epochs=100)

Epoch 1/100
9/9 [==============================] - 1s 20ms/step - loss: 1.6137 - accuracy: 0.3552 - val_loss: 1.6402 - val_accuracy: 0.5345
Epoch 2/100
9/9 [==============================] - 0s 5ms/step - loss: 0.7072 - accuracy: 0.8205 - val_loss: 0.9720 - val_accuracy: 0.7069
Epoch 3/100
9/9 [==============================] - 0s 4ms/step - loss: 0.4112 - accuracy: 0.9151 - val_loss: 0.7044 - val_accuracy: 0.7586
Epoch 4/100
9/9 [==============================] - 0s 4ms/step - loss: 0.2581 - accuracy: 0.9440 - val_loss: 0.5738 - val_accuracy: 0.7931
Epoch 5/100
9/9 [==============================] - 0s 4ms/step - loss: 0.1878 - accuracy: 0.9807 - val_loss: 0.4996 - val_accuracy: 0.8276
Epoch 6/100
9/9 [==============================] - 0s 4ms/step - loss: 0.1545 - accuracy: 0.9846 - val_loss: 0.4409 - val_accuracy: 0.8621
Epoch 7/100
9/9 [==============================] - 0s 5ms/step - loss: 0.1225 - accuracy: 0.9846 - val_loss: 0.3870 - val_accuracy: 0.8793
Epoch 8/100
9/9 [=========

9/9 [==============================] - 0s 3ms/step - loss: 0.0362 - accuracy: 0.9942 - val_loss: 0.0354 - val_accuracy: 0.9828
Epoch 60/100
9/9 [==============================] - 0s 4ms/step - loss: 0.0224 - accuracy: 0.9961 - val_loss: 0.0322 - val_accuracy: 1.0000
Epoch 61/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0273 - accuracy: 0.9961 - val_loss: 0.0273 - val_accuracy: 1.0000
Epoch 62/100
9/9 [==============================] - 0s 4ms/step - loss: 0.0530 - accuracy: 0.9865 - val_loss: 0.0257 - val_accuracy: 1.0000
Epoch 63/100
9/9 [==============================] - 0s 4ms/step - loss: 0.0228 - accuracy: 0.9961 - val_loss: 0.0442 - val_accuracy: 0.9828
Epoch 64/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0306 - accuracy: 0.9884 - val_loss: 0.0505 - val_accuracy: 0.9828
Epoch 65/100
9/9 [==============================] - 0s 4ms/step - loss: 0.0367 - accuracy: 0.9903 - val_loss: 0.0463 - val_accuracy: 0.9828
Epoch 66/100
9/9 [===============

In [16]:
model.evaluate(test_features, test_labels_one_hot)

5/5 [==============================] - 0s 1ms/step - loss: 0.0201 - accuracy: 1.0000


[0.020059330388903618, 1.0]

#Save weights of this model  
model.save_weights('my_model.h5')

#load weights to this TensorFlow model  
model.load_weights('my_model.h5')

In [17]:
# Save model and weights of this model
model.save("model_save")

INFO:tensorflow:Assets written to: model_save\assets


## Reload and preprocess images in TFLM

## Convert model into TFLM format

In [18]:
max_samples = len(test_features)

In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [20]:
test_features = tf.cast(test_features, tf.float32)
tf_lite_ds = tf.data.Dataset.from_tensor_slices((test_features)).batch(1)

def representative_data_gen():
    for input_value in tf_lite_ds.take(100):
        yield [input_value]
    
converter.representative_dataset = representative_data_gen

In [21]:
import pathlib

converter_model = converter.convert()

generated_dir = pathlib.Path("generated/")
generated_dir.mkdir(exist_ok=True, parents=True)
converted_model_file = generated_dir/"pose.tflite"
converted_model_file.write_bytes(converter_model)

INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmpguc_6zzq\assets


D:\Anaconda\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


12664

In order to integrate converted model into TFLM application we have to save it as a C array. One way to do that is to use **xxd** utility available on Linux or in Cygwin/MinGW terminals on Windows. Open terminal and run following commands:

```
cd generated/
xxd -i emnist_model_int8.tflite > model.h
```

The model is ready to be integrated into TFLM application.

## Evaluate TensorFlow Lite Model

In [22]:
import pathlib

generated_dir = pathlib.Path("generated/")
generated_dir.mkdir(exist_ok=True, parents=True)
converted_model_file = generated_dir/"pose.tflite"

interpreter = tf.lite.Interpreter(model_path=str(converted_model_file))
interpreter.allocate_tensors()

# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    scale, zero_point = interpreter.get_output_details()[0]["quantization"]

    prediction_values = []
    
    for test_data in test_features:
        # Pre-processing: add batch dimension, quantize and convert inputs to int8 to match with
        # the model's input data format.
        test_data = np.expand_dims(test_data, axis=0)
        test_data = np.int8(test_data)
        interpreter.set_tensor(input_index, test_data)

        interpreter.invoke()

        # Find the letter with highest probability
        output = interpreter.tensor(output_index)
        result = np.argmax(output()[0])
        prediction_values.append(result)
    
    accurate_count = 0
    for index in range(len(prediction_values)):
        if prediction_values[index] == test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_values)

    return accuracy * 100

Please, keep in mind that full test dataset evaluation on int8 model may take several minutes. 

In [23]:
print(str(evaluate_model(interpreter)) + "%")

100.0%


## Create a test set for target application

In [24]:
import random

# Import training and testing from dataset_buffer
num_of_samples = 25
random_test_features = random.sample(range(1, test_features.shape[0]), num_of_samples)

In [25]:
samples_file = open("generated/test_samples.cpp", "w")

samples_file.write("#include \"test_samples.h\"\n\n")
samples_file.write("const int kNumSamples = " + str(num_of_samples) + ";\n\n")

samples = "" 
samples_array = "const TestSample test_samples[kNumSamples] = {"

for sample_idx, feature_idx in enumerate(random_test_features, 1):
    feature_arr = list(np.array(test_features[feature_idx]).astype("int"))
    var_name = "sample" + str(sample_idx)
    samples += "TestSample " + var_name + " = {\n" #+ "[IMAGE_SIZE] = { "
    samples += "\t.label = " + str(test_labels[feature_idx]) + ",\n" 
    samples += "\t.feature = {\n"
    samples += "\t\t" + str(feature_arr)
    samples += "\t}\n};\n\n"    
    samples_array += var_name + ", "
    
samples = samples.replace("[", "")
samples = samples.replace("]", ",\n")
samples_array += "};\n"

samples_file.write(samples);
samples_file.write(samples_array);
samples_file.close()

## Done

You have converted a Tensorflow model into TFLM format and generated a test set for the application. Now you can copy generated files into target application of this tutorial and try it out:

In order to integrate converted model into TFLM application we have to save it as a C array. One way to do that is to use **xxd** utility available on Linux or in Cygwin/MinGW terminals on Windows. Open terminal and run following commands:

```
cd generated/
xxd -i emnist_model_int8.tflite > model.h
```

The model is ready to be integrated into TFLM application.

* copy *generated/model.h* to *../inc* and *generated/test_samples.cc* to *../src*
* You can start to integrate your WE-I project